### Mongo Spark Assignment

#### Accessing API
- Connecting to Twitter API
- Streaming Tweets for a specific account (could me more)
- Turning steamed tweets output to DataFrame

In [1]:
# Import Dependencies
import tweepy
import pandas as pd
import json

import pymongo
from pymongo import MongoClient

In [2]:
# Authentiation keys and tokens

# Twitter API Keys
ckey='ckey'
csecret='csecret'
atoken='atoken'
asecret='asecret'

# Defining tweepy authentication
auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
# Streaming Timeline tweets for a specific account
stream = api.user_timeline('barstoolbigcat')

Continuous Streaming
http://docs.tweepy.org/en/latest/streaming_how_to.html

In [51]:
# # write to json file
# with open('testdata.json', 'w', encoding='utf-8') as f:
#     json.dump(list_of_dicts, f, ensure_ascii=False, indent=4)

In [4]:
# Creating a list of dicts for a specific twitter accounts timeline
list_of_dicts = []
for each_json_tweet in stream:
    list_of_dicts.append(each_json_tweet._json)

In [5]:
with open('tweet_json_tweets.txt', 'w') as file:
        file.write(json.dumps(list_of_dicts, indent=4))

In [6]:
# Turning streamed tweets into DataFrame
my_demo_list = []
with open('tweet_json_tweets.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = each_dictionary['text']
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        created_at = each_dictionary['created_at']
        my_demo_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'created_at': created_at,
                            })
        #print(my_demo_list)
        tweet_json = pd.DataFrame(my_demo_list, columns = 
                                  ['tweet_id', 'text', 
                                   'favorite_count', 'retweet_count', 
                                   'created_at'])

In [7]:
tweet_json

,tweet_id,text,favorite_count,retweet_count,created_at
0,1249118008242515976,FSU @ Syracuse \n\n#DugsNation \n\nLive Now —&...,8,0,Sat Apr 11 23:31:55 +0000 2020
1,1249114471441879042,Is twitter broken?,813,21,Sat Apr 11 23:17:52 +0000 2020
2,1249112744332128258,RT @3msmith: @BarstoolBigCat Kenny by a nose. ...,0,1,Sat Apr 11 23:11:00 +0000 2020
3,1249111841898258435,Disaster tonight. Some nights the track owns y...,145,1,Sat Apr 11 23:07:25 +0000 2020
4,1249110877409652736,Here’s the periscope. Cat Cave Derby. I don’t ...,190,15,Sat Apr 11 23:03:35 +0000 2020
5,1249110503936290819,IM LIVE ON PERISCOPE SOMEONE SEND ME THE LINK ...,166,0,Sat Apr 11 23:02:06 +0000 2020
6,1249093174473232386,Lanes for tonight’s #CatCaveDerby \n\n1. Kenny...,941,23,Sat Apr 11 21:53:14 +0000 2020
7,1249086491768377352,My stomach hurts,2161,35,Sat Apr 11 21:26:41 +0000 2020
8,1249077629136887808,RT @UGotItJobin: @BarstoolBigCat I didn’t choo...,0,2,Sat Apr 11 20:51:28 +0000 2020
9,1249076612714422272,Tonight - 7:30 pm est\n\nFlorida State @ Syrac...,1527,69,Sat Apr 11 20:47:25 +0000 2020


In [36]:
# Saving tweets to csv
# tweet_json.to_csv("tweetdf.csv", index=False)

#### Connecting to Mongo
- Connecting to MongoClient
- Writing DataFrame to Collection(Tweets), in the 'Twitter' Database

In [38]:
# Making a Connection with MongoClient
client = MongoClient("mongodb://localhost:27017/")
# database
db = client["Twitter"]
# collection
tweets = db["Tweets"]

In [47]:
tweets.insert_many(tweet_json.to_dict("records"))